<a href="https://colab.research.google.com/github/pakrentos/eeg_connectivity/blob/master/psr_mpl_eeg_nikita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!rm -rf /content/src
!git init
!git remote add origin https://github.com/pakrentos/eeg_connectivity/
!git fetch origin
!git checkout origin/master -- src

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 92 (delta 41), reused 35 (delta 14), pack-reused 0
Unpacking objects: 100% (92/92), done.
From https://github.com/pakrentos/eeg_connectivity
 * [new branch]      evgen      -> origin/evgen
 * [new branch]      master     -> origin/master


In [3]:
from src import *
from src.psr import lagged_ami
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from src.psr import global_false_nearest_neighbors, lagged_ami
import warnings
warnings.filterwarnings('ignore')
from statistics import stdev, mean
import json
from src.eeg_utils import r_mapping
from tensorflow import keras

In [14]:
# Поменяйте на вашу папку в диске, где лежат данные субъектов
sub_dir = '/content/drive/My Drive/Subjects'
subject_scores = []
# Поменяйте путь на путь в вашем диске к файлу channels.json
f = open('/content/drive/My Drive/уга-буга/channels.json', 'r')
channels_arr = np.array(json.load(f))
temp_arr = np.arange(465)
nikita_part = temp_arr[:155]
evgen_part = temp_arr[155:310]
pavel_part = temp_arr[310:]

In [15]:
def separate(arr, bins):
    return np.array(np.split(arr, bins))

def prepare(arr):
    arr = np.random.permutation(arr)
    cutoff = arr.shape[0]//2
    train = arr[cutoff:]
    val = arr[:cutoff]
    src = train[:, 0, :]
    trgt = train[:, 1, :]
    src_val = val[:, 0, :]
    trgt_val = val[:, 1, :]
    return src, trgt, src_val, trgt_val

my_callbacks = [
                EarlyStopDifference(patience=10, delta=0.001, verbose=0),
            ]

In [18]:
# Смотрим на всех субъектов
%%time
overall = {}
for sub_num in range(1, 11):
    edata = extract_all(group='YOUNG', hand='righthand', subject=sub_num, sub_dir=sub_dir)
    temp_sub_dict = {}
    print(f'@@@ Subject {sub_num} @@@')
    iter = 0
    # Поменяйте на вашу часть
    for ch_ind in nikita_part:
        ch1 = channels_arr[ch_ind][0]
        ch2 = channels_arr[ch_ind][1]
        print(f'    ### Channel pair {ch1} {ch2} . Num of ch_pair: {iter} ###')
        iter += 1
        ch1_data = select_channel(edata, ch1)
        ch2_data = select_channel(edata, ch2)
        # Фильтруем в нужном нам диапазоне
        lc = 4.0
        hc = 8.0
        ch1_data = butter_bandpass_filter(ch1_data, lc, hc, 250)
        ch2_data = butter_bandpass_filter(ch2_data, lc, hc, 250)
        # Нормализуем
        ch1_data = normalize(ch1_data[:, 250*2:])
        ch2_data = normalize(ch2_data[:, 250*2:])
        # Выбираем коеффициенты для psr. Если коеффициенты не совпадают для каналов на определенном триале -- этот триал откидывается
        coefs_arr = []
        for i in range(len(ch1_data)):
            ch1_lag, ch1_dim = determine_coefs(ch1_data[i], max_lag=10, max_dims=20)
            ch2_lag, ch2_dim = determine_coefs(ch2_data[i], max_lag=10, max_dims=20)
            # print(f'Trial #{i}. {ch1} lag: {ch1_lag}, dims: {ch1_dim}. {ch2} lag: {ch2_lag}, dims: {ch2_dim}')
            if ch1_lag != ch2_lag:
                # print(f'WARNING: Lag of {ch1} does not equal to lag of {ch2}')
                continue
            if ch1_dim != ch2_dim:
                # print(f'WARNING: Num of dims of {ch1} does not equal to number of dims of {ch2}')

                ch1_dim, ch2_dim = max((ch1_dim, ch2_dim)), max((ch1_dim, ch2_dim))
            coefs_arr.append((i, ch1_lag, ch1_dim, ch2_lag, ch2_dim))
        # Эти числа нужны будут для разбивки данных на окна
        bins_arr = np.arange(2, 10)
        scores = []
        for data_ind, ch1_lag, ch1_dim, ch2_lag, ch2_dim in coefs_arr:
            # PSR
            ch1_recon = reconstruct(ch1_data[data_ind], ch1_lag, ch1_dim)
            ch2_recon = reconstruct(ch2_data[data_ind], ch2_lag, ch2_dim)
            # Подготовка данных для запихивания в нейронку
            data = np.stack((ch1_recon, ch2_recon), axis=-2)
            scores_temp = []
            for bins in bins_arr:
                temp = []
                # Разбиение данных на окна
                length = (data.shape[0]//bins)*bins
                # print(f'Trial #{data_ind}. Current bin: {bins}. ', end='')
                for d in separate(data[:length], bins):
                    src, trgt, src_val, trgt_val = prepare(d)
                    model = baseline_model(inputs=data.shape[-1], outputs=data.shape[-1])
                    hist = model.fit(src, trgt, validation_data=(src_val, trgt_val), epochs=50, callbacks=my_callbacks, batch_size=100, verbose=False)
                    r = model.predict(src_val)
                    score = r2_score(trgt_val.T, r.T)
                    temp.append(score)
                scores_mean = np.mean(temp)
                scores_stdev = stdev(temp)
                # print(f'Mean score: {scores_mean}. Stdev: {scores_stdev}')
                scores_temp.append(temp)
            scores.append(scores_temp)
        temp_sub_dict[f'{ch1} {ch2}'] = scores
        overall[sub_num] = temp_sub_dict
        #Поменяйте на ваш путь
        f = open('/content/drive/My Drive/уга-буга/test.json', 'w')
        json.dump(overall, f)
        f.close()

@@@ Subject 1 @@@
    ### Channel pair 17 21 . Num of ch_pair: 0 ###


KeyboardInterrupt: ignored

In [ ]:
class EarlyStopDifference(keras.callbacks.Callback):
    # Custom callback that stops training if the difference between training and validation
    # loss function is more than delta for the past patience training epochs
    
    ### Parameters:
    ### delta: Integer, default=0; Minimal affordable difference between loss functions
    ### patience: Integer, default=0; Number of epochs it is tolerable to have difference greater than delta
    ### verbose: Integer, default=0; Prints output if it is 1
    
    def __init__(self, patience=0, delta=0, verbose=0):
        # Initializing parameters
        super(EarlyStopDifference, self).__init__()
        self.patience=patience
        self.delta=delta
        self.verbose=verbose
        self.counter=0
    
    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        # Recieving loss function values
        mse = logs['loss']
        val_mse = logs['val_loss']
        # Comparing them to delta
        if ((mse - val_mse) <= self.delta):
            # Resetting counter
            self.wait = 0
        else:
            # Incrementing counter
            self.wait += 1
            if self.wait >= self.patience:
                # Stopping the model if wait >= patience
                self.stopped_epoch = epoch
                self.model.stop_training = True
                if (self.verbose != 0):
                    print("Model stopped because mse and val_mse differ for more than ", self.delta, " for the past ", self.patience, " training epochs.")
            
    def on_train_end(self, logs=None):
        # Printing the epoch the model has stopped
        if (self.stopped_epoch > 0) & (self.verbose != 0):
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))

In [17]:
%%time
print('sas')

sas
CPU times: user 911 µs, sys: 24 µs, total: 935 µs
Wall time: 724 µs
